# Run PncA WandB Sweep

In [1]:
from IPython.display import display
import os

if "SSH_CONNECTION" in os.environ:
    display("Running via SSH")
else:
    display("Running locally")
    
import sys
import os

path = os.path.join('..', '/Users/dylandissanayake/Desktop/DPhil/Comp Disc/Repositories/TB-PNCA-GNN') if "SSH_CONNECTION" not in os.environ else os.path.join('..', '/mnt/alphafold-volume-1/dylan2/repos/tb-pnca-gnn')
if path not in sys.path:
    sys.path.append(os.path.abspath(path))

import datetime
import random

import numpy as np
import pandas as pd
import pickle as pkl

import torch
from torch_geometric.data import Data

import wandb

import warnings
warnings.filterwarnings('ignore')

from src import run_model, protein_graph, gcn_model, evaluation

%load_ext autoreload
%autoreload 2

%aimport src

torch.cuda.is_available()

'Running via SSH'

/usr/lib/python3/dist-packages/requests/__init__.py:87: RequestsDependencyWarning: urllib3 (2.2.3) or chardet (4.0.0) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "


True

In [2]:
# with open('datasets/singletons_af_graph_dict.pkl', 'rb') as f:
#     graph_dict = pkl.load(f)
# with open('datasets/singletons_af_w_pza_graph_dict.pkl', 'rb') as f:
#     graph_dict = pkl.load(f)
# with open('datasets/singletons_af_no_mut_feats_graph_dict.pkl', 'rb') as f:
#     graph_dict = pkl.load(f)

with open('datasets/x19_exp_l2_graph_dict.pkl', 'rb') as f:
    graph_dict = pkl.load(f)

In [3]:
len(graph_dict['train']) + len(graph_dict['test'])

664

### Set Up Params and Sweep Config

In [4]:
seed = 42
np.random.seed(seed)
random.seed(seed)

# logging params (only used for wandb metrics)
n_samples = len(graph_dict['train']) + len(graph_dict['test'])
# cutoff_distance = 6.3  

# gcn params
# num_node_features = 16
num_node_features = 19
batch_size = 256
# hidden_channels = 64
# learning_rate = 0.001
# wd = 5e-5
epochs = 2000

wt_seq = 'MRALIIVDVQNDFCEGGSLAVTGGAALARAISDYLAEAADYHHVVATKDFHIDPGDHFSGTPDYSSSWPPHCVSGTPGADFHPSLDTSAIEAVFYKGAYTGAYSGFEGVDENGTPLLNWLRQRGVDEVDVVGIATDHCVRQTAEDAVRNGLATRVLVDLTAGVSADTTVAALEEMRTASVELVCS'

In [5]:
# # First sweep:

# sweep_config = {
#     'method': 'random'
#     }

# metric = {
#     'name': 'Test Accuracy',
#     'goal': 'maximize'   
#     }

# sweep_config['metric'] = metric

# parameters_dict = {
#     'hidden_channels': {
#         'values': [64, 128, 192, 256, 320, 384]
#         },
#     'weight_decay': {
#         'distribution': 'log_uniform_values',
#         'min': 1e-8,
#         'max': 1e-2
#         },
#     'dropout': {
#           'values': [0.2, 0.4, 0.5, 0.6, 0.8]
#         },
#     'cutoff_distance': {
#         'distribution': 'uniform',
#         'min': 4,
#         'max': 12 
#         },
#     'learning_rate': {
#         'distribution': 'log_uniform_values',
#         'min': 1e-8,
#         'max': 1e-1
#       },
#     }

# sweep_config['parameters'] = parameters_dict

# # Second sweep:

# parameters_dict.update({
#     'learning_rate': {
#         'distribution': 'log_uniform_values',
#         'min': 1e-5,
#         'max': 1e-2 
#         },
#     'hidden_channels': {
#         'values': [128, 192, 256, 320, 384]
#         },
#     })

# # Third sweep:

# parameters_dict.update({
#     'learning_rate': {
#         'distribution': 'log_uniform_values',
#         'min': 1e-5,
#         'max': 1e-3 
#         },
#     'edge_weights': {
#         'values': ["dist", "1-(dist/cutoff)", "1/dist"]
#         },
#     })

# # Fourth sweep:

# parameters_dict.update({
#     'edge_weights': {
#         'values': ["dist", "1-(dist/cutoff)", "1/dist", "l=0.1", "l=0.2", "l=0.3", "l=0.4", "l=0.5", "l=0.8", "l=1.0", "l=1.2", "l=1.5", "l=2.0"]
#         },
#     })


In [6]:
sweep_config = {
'method': 'random'
}

metric = {
    'name': 'Test Accuracy',
    'goal': 'maximize'   
    }

sweep_config['metric'] = metric

parameters_dict = {
    'hidden_channels': {
        'values': [128, 192, 256, 320, 384]
        },
    'weight_decay': {
        'distribution': 'log_uniform_values',
        'min': 1e-8,
        'max': 1e-2
        },
    'dropout': {
        'values': [0.2, 0.4, 0.5, 0.6, 0.8]
        },
    # 'cutoff_distance': {
    #     'distribution': 'uniform',
    #     'min': 6,
    #     'max': 15 
    #     },
    'learning_rate': {
        'distribution': 'log_uniform_values',
        'min': 1e-6,
        'max': 1e-2
    },
    # 'lambda_param': {
    #     'values': [0.2, 0.5, 1.0, 1.5, 2.0, 2.5, 3.0]
    #     },
    }

sweep_config['parameters'] = parameters_dict

In [7]:
import pprint
pprint.pprint(sweep_config)

{'method': 'random',
 'metric': {'goal': 'maximize', 'name': 'Test Accuracy'},
 'parameters': {'dropout': {'values': [0.2, 0.4, 0.5, 0.6, 0.8]},
                'hidden_channels': {'values': [128, 192, 256, 320, 384]},
                'learning_rate': {'distribution': 'log_uniform_values',
                                  'max': 0.01,
                                  'min': 1e-06},
                'weight_decay': {'distribution': 'log_uniform_values',
                                 'max': 0.01,
                                 'min': 1e-08}}}


### Define Training Loop

In [8]:
# project = "pnca-af-singletons-sweep-w-mutation-feats"
# project = "pnca-af-singletons-sweep-NO-mutation-feats"
# project = "pnca-af-singletons-sweep-w-mutation-feats-exp-edge-weights"

project = "19-node-feats"

In [9]:
sweep_id = wandb.sweep(sweep_config, project=project)
## project 1
# sweep_id = 'x0k5kbpt'
# sweep_id = 'mnkuowkk'
# sweep_id = 'jp3xzyp5'
#! sweeps from here include proper pza distance
# sweep_id = '9b3bkt5y'
# sweep_id = 'w1bgy5pd'
# sweep_id = 'r7865la3'

# sweep_id = wandb.sweep(sweep_config, project=project)
# sweep_id = 'fypk3i0y'

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
/usr/lib/python3/dist-packages/requests/__init__.py:87: RequestsDependencyWarning: urllib3 (2.2.3) or chardet (4.0.0) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "


Create sweep with ID: uuv8rla0
Sweep URL: https://wandb.ai/dylan-home/19-node-feats/sweeps/uuv8rla0


In [ ]:
# run_path = "dylan-home/19-node-feats/431g130a"
# api = wandb.Api()
# best_conf = api.run(run_path).config

In [ ]:
# best_conf['learning_rate'] = 5e-4

# best_conf

{'dropout': 0.6,
 'lambda_param': 2,
 'weight_decay': 0.004281527452568532,
 'learning_rate': 0.0005,
 'cutoff_distance': 11.74916595319417,
 'hidden_channels': 320}

In [24]:
# Run a single run

# cutoff_distance=12.224281635103226
# dropout=0.4

# edge_weights="exp"
# hidden_channels=320
# lambda_param=1.9594
# learning_rate=0.000043988
# weight_decay=8.2056e-7

# params = {
#   "dropout": 0.4,
#   "edge_weights": "exp",
#   "lambda_param": 1.9593979073129404,
#   "weight_decay": 8.205598117080135e-7,
#   "learning_rate": 0.000043987923400618824,
#   "cutoff_distance": 12.224281635103226,
#   "hidden_channels": 320
# }


model = run_model.pnca_GCN_vary_graph(
            self_loops = False,
            cutoff_distance = best_conf['cutoff_distance'],
            edge_weight_func = 'exp',
            batch_size = batch_size,
            num_node_features = num_node_features,
            hidden_channels = best_conf['hidden_channels'],
            learning_rate = best_conf['learning_rate'],
            wd = best_conf['weight_decay'],
            dropout = best_conf['dropout'],
            lr_scheduling=False,
            epochs = epochs,
            graph_dict= graph_dict,
            normalise_ews=True,
            lambda_param= best_conf['lambda_param'],
            wandb_params={
              'use_wandb': True, 
              'wandb_project': f'{project}', 
              'wandb_name': 'run 5 light GraphNorm1+2',
              'n_samples': n_samples,
              'sweep': False
              }
        )

Adjusting edge index and attaching edge weights for cutoff distance 11.74916595319417
Using CUDA


Early stopping enabled. Patience: 50. Min Delta: 0.
Epoch: 000, Train Acc: 0.5302, Test Acc: 0.5150, Train Loss: 0.7103, Test Loss: 0.7182
Epoch: 010, Train Acc: 0.5948, Test Acc: 0.5900, Train Loss: 0.6782, Test Loss: 0.6767
Epoch: 020, Train Acc: 0.6897, Test Acc: 0.6550, Train Loss: 0.6402, Test Loss: 0.6429
Epoch: 030, Train Acc: 0.7414, Test Acc: 0.6750, Train Loss: 0.5773, Test Loss: 0.5958
Epoch: 040, Train Acc: 0.7500, Test Acc: 0.6900, Train Loss: 0.5347, Test Loss: 0.5593
Epoch: 050, Train Acc: 0.7909, Test Acc: 0.6950, Train Loss: 0.4887, Test Loss: 0.5463
Epoch: 060, Train Acc: 0.7974, Test Acc: 0.7050, Train Loss: 0.4663, Test Loss: 0.5538
Epoch: 070, Train Acc: 0.7909, Test Acc: 0.7150, Train Loss: 0.4728, Test Loss: 0.5986
Epoch: 080, Train Acc: 0.7974, Test Acc: 0.7200, Train Loss: 0.4423, Test Loss: 0.5234
Epoch: 090, Train Acc: 0.8190, Test Acc: 0.7350, Train Loss: 0.4352, Test Loss: 0.5500
Epoch: 100, Train Acc: 0.8147, Test Acc: 0.7300, Train Loss: 0.4165, Test Loss

wandb: WARNING Source type is set to 'repo' but some required information is missing from the environment. A job will not be created from this run. See https://docs.wandb.ai/guides/launch/create-job


Test Accuracy,▁▃▄▅▅▆▅▅▆▆▆▆▆▆▇▇▇▇▇▇▇▇▇██▇██████▇▇██▇▇▇▇
Test F1,▃▁▂▄▇▅▂▁▃▄▄▄▅▅▅▇▆▆▆▆▆▆▆▇▇▆▇▇▇▇▇█▆▆▇▇▆▆▆▆
Test Loss,██▇▇▆▅▄▄▃▃▃▂▂▂▄▁▂▂▂▂▂▂▃▁▁▂▁▂▂▂▁▁▃▃▂▂▃▄▄▅
Test Sensitivity,▇▃▃▅█▄▁▁▂▂▂▃▃▃▂▅▄▃▃▄▄▃▃▄▄▄▄▅▅▄▅▅▄▄▅▄▄▃▄▃
Test Specificity,▁▅▆▅▃▆▇█▇▇█▇▇▇█▇▇▇█▇▇██▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇
Train Accuracy,▁▁▃▄▃▅▅▅▆▆▆▆▆▆▇▆▆▇▇▇▇▇▇▆▇▇▇▇▇▇▇▆▇▇▇▇████
Train F1,▃▁▃▄▅▆▅▅▆▆▆▆▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇██▇█████
Train Loss,███▇▇▆▆▅▅▄▄▄▄▄▄▃▃▃▃▃▃▃▃▃▃▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁
Train Sensitivity,▆▁▂▅█▆▄▃▄▄▄▅▅▅▄▆▆▅▅▅▅▅▅▆▆▆▆▆▆▆▇▆▆▆▇▆▆▅▆▅
Train Specificity,▁▄▅▄▁▅▇▇▇▇▇▆▆▆█▆▆▇▇▇▇▇▇▆▆▇▆▇▆▆▆▆▇▇▆▇▇█▇█
Test Accuracy,0.73


In [ ]:
# os.makedirs(f'saved_models/carter_ds_aug/{project}/8hxle9sc', exist_ok=True)
        
# torch.save(model, f'saved_models/carter_ds_aug/{project}/8hxle9sc/hopeful-sweep-190')

In [10]:
for t in graph_dict:
    for sample in graph_dict[t]:
        col_select = [i for i in range(19) if i != 14]
        new_feats = graph_dict[t][sample]['graph'].dataset[0].x[:, col_select]
        
        graph_dict[t][sample]['graph'].dataset[0].x = new_feats
        

In [11]:
def sweep_run():

    with wandb.init() as run:
        config = run.config

        model = run_model.pnca_GCN_vary_graph(
            self_loops = False,
            # cutoff_distance = config.cutoff_distance,
            cutoff_distance = 12,
            edge_weight_func = 'exp',
            batch_size = batch_size,
            # num_node_features = num_node_features,
            num_node_features = 18,
            hidden_channels = config.hidden_channels,
            learning_rate = config.learning_rate,
            wd = config.weight_decay,
            dropout = config.dropout,
            lr_scheduling=False,
            epochs = epochs,
            graph_dict= graph_dict,
            normalise_ews=True,
            # lambda_param=config.lambda_param,
            lambda_param=2,
            wandb_params={
                'use_wandb': False,
                'sweep': True
            }
        )

        # os.makedirs(f'saved_models/carter_ds_aug/{project}/{sweep_id}', exist_ok=True)
        
        # torch.save(model, f'saved_models/carter_ds_aug/{project}/{sweep_id}/{run.name}')

In [12]:
wandb.agent(sweep_id, sweep_run, project = project, count=100)

wandb: Agent Starting Run: 9u2t0od4 with config:
wandb: 	dropout: 0.2
wandb: 	hidden_channels: 256
wandb: 	learning_rate: 1.4130211118325083e-06
wandb: 	weight_decay: 0.004071546397518974
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
/usr/lib/python3/dist-packages/requests/__init__.py:87: RequestsDependencyWarning: urllib3 (2.2.3) or chardet (4.0.0) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "
wandb: Currently logged in as: dylan-home. Use `wandb login --relogin` to force relogin


Using CUDA
Early stopping enabled. Patience: 50. Min Delta: 0.
Epoch: 000, Train Acc: 0.4698, Test Acc: 0.4850, Train Loss: 0.6963, Test Loss: 0.6952
Epoch: 010, Train Acc: 0.4698, Test Acc: 0.4850, Train Loss: 0.6991, Test Loss: 0.6967
Epoch: 020, Train Acc: 0.4698, Test Acc: 0.4850, Train Loss: 0.6989, Test Loss: 0.6970
Epoch: 030, Train Acc: 0.4698, Test Acc: 0.4850, Train Loss: 0.6975, Test Loss: 0.6962
Epoch: 040, Train Acc: 0.4698, Test Acc: 0.4850, Train Loss: 0.6961, Test Loss: 0.6951
Epoch: 050, Train Acc: 0.4720, Test Acc: 0.4850, Train Loss: 0.6945, Test Loss: 0.6942
Epoch: 060, Train Acc: 0.4612, Test Acc: 0.4950, Train Loss: 0.6936, Test Loss: 0.6937
Epoch: 070, Train Acc: 0.5237, Test Acc: 0.5000, Train Loss: 0.6928, Test Loss: 0.6933
Epoch: 080, Train Acc: 0.5302, Test Acc: 0.5150, Train Loss: 0.6924, Test Loss: 0.6931
Epoch: 090, Train Acc: 0.5302, Test Acc: 0.5150, Train Loss: 0.6922, Test Loss: 0.6930
Epoch: 100, Train Acc: 0.5302, Test Acc: 0.5150, Train Loss: 0.6919

Test Accuracy,▂▁▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇█▇▇▇▇▇▇▇▇▇▇▇▇
Test F1,▁▃██████████████████████████████████████
Test Loss,█▅▅▅▅▅▄▄▄▄▄▄▄▄▄▃▃▃▃▃▃▃▃▃▃▃▂▂▂▂▂▂▂▂▂▁▁▁▁▁
Test Sensitivity,▁▂██████████████████████████████████████
Test Specificity,█▇▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
Train Accuracy,▁▂▆▆▆▆▆▆▆▆▆▆▆▆▆▆▆▆▆▆▆▆▆▆▆▆▆▆▇██▇██▇█████
Train F1,▁▄██████████████████████████████████████
Train Loss,█▅▄▄▄▄▄▄▄▃▃▃▃▃▃▃▃▃▃▃▃▃▂▂▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁
Train Sensitivity,▁▂██████████████████████████████████████
Train Specificity,█▇▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▂▁▂▂▂▂▂▂▂▂
Test Accuracy,0.52


wandb: Agent Starting Run: hav5iwwh with config:
wandb: 	dropout: 0.2
wandb: 	hidden_channels: 192
wandb: 	learning_rate: 3.608231051322591e-06
wandb: 	weight_decay: 0.0007307800797066199
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
/usr/lib/python3/dist-packages/requests/__init__.py:87: RequestsDependencyWarning: urllib3 (2.2.3) or chardet (4.0.0) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "


Using CUDA
Early stopping enabled. Patience: 50. Min Delta: 0.
Epoch: 000, Train Acc: 0.5302, Test Acc: 0.5150, Train Loss: 0.6913, Test Loss: 0.6933
Epoch: 010, Train Acc: 0.5323, Test Acc: 0.5100, Train Loss: 0.6924, Test Loss: 0.6932
Epoch: 020, Train Acc: 0.5323, Test Acc: 0.5100, Train Loss: 0.6922, Test Loss: 0.6933
Epoch: 030, Train Acc: 0.5302, Test Acc: 0.5100, Train Loss: 0.6917, Test Loss: 0.6933
Epoch: 040, Train Acc: 0.5302, Test Acc: 0.5150, Train Loss: 0.6915, Test Loss: 0.6933
Epoch: 050, Train Acc: 0.5302, Test Acc: 0.5150, Train Loss: 0.6918, Test Loss: 0.6933
50 epochs passed without 0 test loss improvement. 
Early stopping triggered.


...

Test Accuracy,▁▁▁▁▁▁▁▂▄▅▅▄▄▄▅▅▅▅▅▆▅▅▇▆▆▆▇▇▇▆▇█▇▆█▆▆█▇▇
Test F1,▅▅▅▅▅▅▅▅▄▅▃▁▂▂▄▃▄▅▄▇▅▄▇▅▅▆▆▇▆▆▇██▅█▅▄█▆▇
Test Loss,███████▇▇▇▆▆▅▄▄▄▄▃▃▃▂▄▂▂▂▂▁▁▁▁▁▁▁▂▁▂▅▁▁▁
Test Sensitivity,███████▇▄▄▂▁▂▂▃▂▂▄▃▅▃▂▅▃▄▄▄▅▄▄▄▅▆▃▅▃▃▅▄▄
Test Specificity,▁▁▁▁▁▁▂▃▆▆▇███▇██▇█▇▇█▇▇▇▇▇▇███▇▆█▇██▇██
Train Accuracy,▁▁▁▁▂▂▃▃▅▅▅▄▅▅▆▅▆▇▆▇▇▆▇▇▇▇██▇▇██▇▇█▇▆███
Train F1,▄▄▄▄▄▅▅▅▄▄▄▁▃▃▅▄▄▆▅▇▆▄▇▆▇▇▇▇▇▇▇██▆█▆▄███
Train Loss,████████▇▇▇▆▆▅▅▅▄▄▄▃▃▄▃▃▂▂▂▂▂▂▂▁▁▂▁▂▄▁▂▁
Train Sensitivity,███████▇▄▃▃▁▂▂▃▃▃▄▃▅▄▂▅▄▄▅▅▅▅▄▅▅▆▄▅▄▂▆▅▅
Train Specificity,▁▁▁▁▂▂▂▃▆▇▇█▇█▇██▇█▇▇█▇██▇▇▇███▇▇█▇██▇██
Test Accuracy,0.77


wandb: Agent Starting Run: xrc6lusm with config:
wandb: 	dropout: 0.6
wandb: 	hidden_channels: 192
wandb: 	learning_rate: 2.8795466695938132e-05
wandb: 	weight_decay: 5.553849691762898e-08
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
/usr/lib/python3/dist-packages/requests/__init__.py:87: RequestsDependencyWarning: urllib3 (2.2.3) or chardet (4.0.0) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "


Using CUDA
Early stopping enabled. Patience: 50. Min Delta: 0.
Epoch: 000, Train Acc: 0.5302, Test Acc: 0.5150, Train Loss: 0.6913, Test Loss: 0.6934
Epoch: 010, Train Acc: 0.5302, Test Acc: 0.5150, Train Loss: 0.6914, Test Loss: 0.6931
Epoch: 020, Train Acc: 0.5302, Test Acc: 0.5150, Train Loss: 0.6909, Test Loss: 0.6926
Epoch: 030, Train Acc: 0.5302, Test Acc: 0.5150, Train Loss: 0.6900, Test Loss: 0.6921
Epoch: 040, Train Acc: 0.5302, Test Acc: 0.5150, Train Loss: 0.6895, Test Loss: 0.6920
Epoch: 050, Train Acc: 0.5280, Test Acc: 0.5150, Train Loss: 0.6896, Test Loss: 0.6916
Epoch: 060, Train Acc: 0.5280, Test Acc: 0.5150, Train Loss: 0.6886, Test Loss: 0.6914
Epoch: 070, Train Acc: 0.5280, Test Acc: 0.5150, Train Loss: 0.6886, Test Loss: 0.6907
Epoch: 080, Train Acc: 0.5280, Test Acc: 0.5150, Train Loss: 0.6884, Test Loss: 0.6908
Epoch: 090, Train Acc: 0.5280, Test Acc: 0.5150, Train Loss: 0.6885, Test Loss: 0.6905
Epoch: 100, Train Acc: 0.5453, Test Acc: 0.5200, Train Loss: 0.6877

wandb: Ctrl + C detected. Stopping sweep.

Aborted!


Epoch: 1440, Train Acc: 0.8233, Test Acc: 0.7550, Train Loss: 0.4195, Test Loss: 0.5285
Epoch: 1450, Train Acc: 0.8233, Test Acc: 0.7600, Train Loss: 0.4144, Test Loss: 0.5274
